# 🧠 Bayesian-AI Full Learning Cycle Dashboard

**Objective:** Execute the full learning cycle on all available data with real-time visualization of P&L and Confidence.

**Requirements:**
*   **CUDA Required:** This notebook will fail if CUDA is not available.
*   **Data:** Loads all `.dbn` and `.parquet` files from `DATA/RAW`.


In [ ]:
import sys
import os
import glob
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from numba import cuda

# Add root to path
current_dir = os.path.abspath('')
project_root = os.path.dirname(current_dir) if 'notebooks' in current_dir else current_dir
if project_root not in sys.path:
    sys.path.append(project_root)

from training.orchestrator import TrainingOrchestrator, load_data_from_directory, get_data_source
from config.settings import RAW_DATA_PATH

print(f"Project Root: {project_root}")
print(f"Data Path: {RAW_DATA_PATH}")

## 1. Strict CUDA Enforcement 🛑
The full learning cycle requires GPU acceleration. CPU fallback is strictly disabled.

In [ ]:
try:
    if not cuda.is_available():
        raise RuntimeError("❌ CRITICAL: CUDA Required for Full Learning Cycle. CPU Fallback is disabled.")
    
    device = cuda.get_current_device()
    print(f"🟢 CUDA Active: {device.name}")
    print(f"   Compute Capability: {device.compute_capability}")
    
except ImportError:
    raise RuntimeError("❌ CRITICAL: Numba CUDA module not found. Please install CUDA drivers and Numba.")
except Exception as e:
    raise RuntimeError(f"❌ CRITICAL: CUDA Check Failed: {e}")

## 2. Load Training Data 📂
Loading all historical data from `DATA/RAW`.

In [ ]:
try:
    # Check if DATA/RAW exists
    data_path = os.path.join(project_root, 'DATA', 'RAW')
    if not os.path.exists(data_path):
        # Fallback for some envs
        data_path = 'DATA/RAW'
        
    print(f"Loading data from: {data_path}...")
    full_data = load_data_from_directory(data_path)
    print(f"🟢 Data Loaded. Rows: {len(full_data):,}")
    
    # Preview
    display(full_data.head())
    
except Exception as e:
    print(f"🔴 Data Load Error: {e}")
    # Stop execution if no data
    raise e

## 3. Learning Dashboard 📊
Real-time visualization of P&L and Confidence.

In [ ]:
# Setup Widgets
fig = go.FigureWidget()
fig.add_scatter(name="Daily PnL", y=[], mode='lines+markers', line=dict(color='green'))
fig.add_scatter(name="Avg Confidence", y=[], mode='lines', line=dict(color='blue', dash='dot'), yaxis='y2')

fig.update_layout(
    title="Learning Progress: PnL & Confidence",
    xaxis_title="Iteration",
    yaxis_title="PnL ($)",
    yaxis2=dict(
        title="Avg Confidence",
        overlaying='y',
        side='right',
        range=[0, 1]
    ),
    height=500,
    template="plotly_dark"
)

progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    style={'bar_color': '#00ff00'},
    layout=widgets.Layout(width='100%')
)

status_label = widgets.Label(value="Ready to start.")
stats_output = widgets.Output()

display(widgets.VBox([progress_bar, status_label, fig, stats_output]))

def update_dashboard(metrics):
    # Update Charts
    with fig.batch_update():
        fig.data[0].y = list(fig.data[0].y) + [metrics['pnl']]
        fig.data[1].y = list(fig.data[1].y) + [metrics['average_confidence']]
        fig.data[0].x = list(range(1, len(fig.data[0].y) + 1))
        fig.data[1].x = list(range(1, len(fig.data[1].y) + 1))

    # Update Progress
    progress_bar.value = metrics['iteration']
    progress_bar.max = metrics['total_iterations']
    
    status_label.value = f"Iter {metrics['iteration']}/{metrics['total_iterations']} | PnL: ${metrics['pnl']:.2f} | WR: {metrics['win_rate']:.1%} | Conf: {metrics['average_confidence']:.2f}"
    
    # Log to output (optional, keeping it clean)
    # with stats_output:
    #     print(f"Iter {metrics['iteration']}: PnL=${metrics['pnl']:.2f}, Conf={metrics['average_confidence']:.3f}")

## 4. Execute Full Learning Cycle 🚀
Run the training loop.

In [ ]:
ITERATIONS = 50 # Default for full cycle, user can adjust

btn_start = widgets.Button(description="Start Training", button_style='success', icon='play')

def start_training(b):
    btn_start.disabled = True
    status_label.value = "Initializing Orchestrator..."
    
    try:
        # Clear previous data
        with fig.batch_update():
            fig.data[0].y = []
            fig.data[1].y = []
        
        # Initialize Orchestrator
        # output_dir='models/production_learning' to separate from debug
        orch = TrainingOrchestrator(
            asset_ticker='MNQ', 
            data=full_data, 
            output_dir='models/production_learning',
            use_gpu=True, # ENFORCE GPU
            verbose=False
        )
        
        status_label.value = "Training Started..."
        
        # Run
        orch.run_training(iterations=ITERATIONS, on_progress=update_dashboard)
        
        status_label.value = "✅ Training Complete! Model Saved."
        progress_bar.bar_style = 'success'
        
    except Exception as e:
        status_label.value = f"❌ Error: {str(e)}"
        progress_bar.bar_style = 'danger'
        with stats_output:
            print(f"Training Exception: {e}")
            import traceback
            traceback.print_exc()
    finally:
        btn_start.disabled = False

btn_start.on_click(start_training)
display(btn_start)